In [6]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

df = pd.read_csv('joined_dataset.csv', low_memory=False)

reader = Reader(rating_scale=(1, 10))  # Adjust the rating scale if different
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)

model = SVD()

model.fit(trainset)

predictions = model.test(testset)


In [7]:
from surprise import accuracy

mean_squared_error = accuracy.rmse(predictions)

accuracy.rmse(predictions)

RMSE: 1.6325
RMSE: 1.6325


np.float64(1.6325221144787343)

In [8]:
from sklearn.metrics import mean_squared_error

# Assuming df contains columns 'UserID', 'ISBN', and 'Rating'
mean_rating = df['Book-Rating'].mean()

# Use the mean rating as the baseline prediction for all rows
baseline_predictions = np.full(shape=len(df), fill_value=mean_rating)

# Calculate the RMSE between the actual ratings and the baseline predictions
baseline_rmse = np.sqrt(mean_squared_error(df['Book-Rating'], baseline_predictions))

print(f'Baseline RMSE: {baseline_rmse}')

Baseline RMSE: 1.8413257666186538


In [9]:
import pickle

# Save the model
with open('svd_model.pkl', 'wb') as file:
    pickle.dump(model, file)
